In [1]:
import scipy.io
from tables import *
import pandas as pd
import os
import shutil
from pathlib import Path
import tqdm
from tqdm.notebook import tqdm
from directify import directify
from random import sample
from PIL import Image

In [2]:
tqdm.pandas()

In [4]:
PATH_TO_META = "<REPLACE WITH YOUR PATH>"

data = scipy.io.loadmat(PATH_TO_META)
lean = data['imdb'][0][0]

def recruit_images(df,size):
    """ lean is an array of arrays from the imdb_crop.tar file from the IMDB faces dataset. 
         Size is the minimum number of images from each class (name).
         Writes images to a "./selected/" directory."""
    SIZE = size
    names = df["Name"].tolist()
    unique = df["Name"].unique()
    paths = df["Path"].tolist()
    joined = list(zip(names,paths))
    
    for name in tqdm(unique):
        #print(name)
        valid = [item[1] for item in joined if item[0] == name]
        # print("valid",valid)
        #print(valid[:10])
        sampled_valid_data = [item for item in sample(valid,SIZE)]
        # print(sampled_valid_data[:10])
        directify(sampled_valid_data)
    print("Recruited!")

def add_image_dimensions(df, image_path_column):
    """adds two columns to the dataframe df that are the width and height of the image in df["Path"]"""
    def get_dimensions(row):
        path=row["Path"]
        try:
            with Image.open(path) as img:
                return img.size
        except Exception as e:
            print(f"Error processing image at {path}: {str(e)}")
            return (None, None)
    
    # Create new column names
    width_column = f"{image_path_column}_width"
    height_column = f"{image_path_column}_height"

    dimensions = df.progress_apply(func=get_dimensions,axis=1)
    print(dimensions)
    df[width_column] = [item[0] for item in dimensions]
    df[height_column] = [item[1] for item in dimensions]
    return df

FileNotFoundError: [Errno 2] No such file or directory: 'imdb.mat'

In [5]:
# the peoples' names are in lean[4] and their corresponding image paths are in lean[2]
names = [item[0] for item in lean[4][0]]
paths = [item[0] for item in lean[2][0]]
joined = list(zip(names,paths))
counts = pd.Series([item[0] for item in joined]).value_counts()
freq = counts[counts >= 50].index.tolist()
frequent_fliers = [item for item in joined if item[0] in freq]

df = pd.DataFrame({
    "Name":[item[0] for item in lean[4][0]],
    "Path":["./imdb_crop/"+item[0] for item in lean[2][0]],
    "Gender":lean[3][0],
    "Confidence_1":lean[6][0],
    "Confidence_2":lean[7][0],
                  })

NameError: name 'lean' is not defined

In [3]:
print(f"Here is a value count of the number of images associated with each name {counts}")
print(f"Here is a list of the people with at least 50 images in the image set {frequent_fliers}")

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


NameError: name 'counts' is not defined

In [6]:
confident_df = df[(df['Confidence_1']>=0.0)]
#I tried filtering on confidence, however the resulting number of images was too low for contrastive learning
conf_count = confident_df["Name"].value_counts()
frequent_confident = conf_count[conf_count >= 50].index.tolist()
len(df[df["Name"].isin(frequent_confident)]),len(frequent_confident)
#While the overall corpus seems sufficiently large, I am concerned with a sub-100,000 images dataset. 
#I would rather use a larger but less precise dataset.

(298777, 1975)

In [7]:
df_dim = add_image_dimensions(df,"Path")
df_dim.to_csv("imdb_crop_dimensions.csv",index=False)

  0%|          | 0/460723 [00:00<?, ?it/s]

0         (257, 257)
1         (263, 263)
2         (500, 500)
3         (400, 401)
4         (340, 340)
             ...    
460718    (157, 157)
460719    (333, 500)
460720    (354, 500)
460721    (292, 292)
460722    (111, 111)
Length: 460723, dtype: object


In [8]:
df_dim = pd.read_csv("imdb_crop_dimensions.csv")
big_images = df_dim[(df_dim["Path_height"]>100) & (df_dim["Path_width"]>100)]
big_counts = big_images["Name"].value_counts()
freq = big_counts[big_counts >= 50].index.tolist()
freq_big_images = big_images[big_images["Name"].isin(freq)]
len(freq_big_images),len(freq)

(272378, 2055)

In [9]:
recruit_images(freq_big_images,50)

  0%|          | 0/2055 [00:00<?, ?it/s]


Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to selected

Copied 50 files successfully to s